# Link: https://patches12306.github.io/patches12306/


## Garrett Van Beek
## Ben Nguyen
## Intro to Data Science
## Professor Nick Mattei

# Final Tutorial Milestone 1: “Can you Google the stock market?"

For our project we plan to collect data on search traffic from Google and Twitter and see how that data correlates with the performances of certain publicly traded stocks. For example, one question we would like to ask is “how does the search traffic about clean energy sources correlate with the performance of clean energy stocks?”. We would also like to see how the sentiment about certain topics correlates to their stock. For this, we plan to use a python library Twitter Sentiment Analysis or Tweepy. This tool gives us a sentiment rating that we can compare to our stock’s performance. We will most likely analyze our data as line graphs. This way we can see how stock performance, search traffic, and sentiment change over time. Consequently, we hope to find correlations and gain insight on the data.

We hope to tackle topics that are a bit more controversial or nuanced than clean energy. Some of the questions that we have thought of asking are: “How does search traffic on sex scandals within companies correlate to the stocks of the companies?” We can also observe the effects of influencers like Donald Trump or Kanye West have on the stock market. We can observe their twitter comments about certain topics and then observe how stocks relating to these topic’s perform. Another angle we can observe is how the general sentiment on topics will correlate to related stocks. For example, we can ask “How does the sentiment about Elon Musk correlate to the stock performance of TESLA?”

For our information on stock performance we plan on scraping or downloading data from Yahoo Finance, which contains a barrage of data on publicly traded stocks. Links to the datasets, tools, and websites we plan to use are provided below.

https://trends.google.com/trends/?geo=US https://www.tweepy.org/ https://github.com/shaypal5/awesome-twitter-data https://finance.yahoo.com/

tutorial project for datascience

# Final Tutorial Milestone 2

## Questions
Our overall goal is to determine "Can we google the stock market?" By this we mean "can we find correlations between google search trends/twitter activity on certain topics and the performance of stocks related to those topics?"

This is a very general questions so we will answer two specific questions based on our search history. 

### 1. How does Elon Musk's Twitter activity correlate to the stock performance of Tesla. How do search trends on Elon Musk correlate to the performance of Tesla Stock?
Note: we are currently waiting for authorization to use Twitter's Tweepy API. Until then, we will be using google trend data from Google Trends.

#### Step One: Get the Data
We download google search data as CSVs. Our goal is to create a DataFrame where every day is a row, and the columns are:
1. Search trends of Elon Musk
2. Performance of TESLA Stock
3. Performance of PAYPAL Stock

Our timeframe for these questions will be 2016-2018

In [1]:
#Import Statements
import pandas as pd

In [73]:
#Read csv into dataframe
tesla_df = pd.read_csv('./data/tesla2016-2018.csv')
paypal_df = pd.read_csv('./data/paypal2016-2018.csv')
# elon_df = pd.read_csv('./data/elon2016-2018.csv')

# tesla_df
# paypal_df
# elon_df["Date"] = elon_df["Week"]
# elon_df

# tesla_df["Date"]
joined_df = pd.merge(tesla_df, paypal_df, on="Date", how="inner", suffixes=('_t','_p'))
# complete_df = pd.merge(joined_df, elon_df, on="Date", how="left")
# complete_df["Week"].count()
# complete_df
joined_df

# filter the data so the only columns are date, closing price of tesla and paypal
# slim_df = joined_df[["Date", "Close_t", "Close_p"]]
# slim_df


,Date,Open_t,High_t,Low_t,Close_t,Adj Close_t,Volume_t,Open_p,High_p,Low_p,Close_p,Adj Close_p,Volume_p
0,2016-01-04,230.720001,231.380005,219.000000,223.410004,223.410004,6827100,35.130001,35.560001,34.279999,34.750000,34.750000,12287700
1,2016-01-05,226.360001,226.889999,220.000000,223.429993,223.429993,3186800,34.980000,34.980000,33.860001,34.310001,34.310001,11227700
2,2016-01-06,220.000000,220.050003,215.979996,219.039993,219.039993,3779100,33.700001,34.009998,33.209999,33.980000,33.980000,8441300
3,2016-01-07,214.190002,218.440002,213.669998,215.649994,215.649994,3554300,33.150002,34.160999,33.020000,33.130001,33.130001,11041100
4,2016-01-08,217.860001,220.440002,210.770004,211.000000,211.000000,3628100,33.459999,33.880001,32.630001,32.689999,32.689999,7848800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,2018-12-24,313.500000,314.500000,295.200012,295.390015,295.390015,5559900,77.480003,79.320000,76.699997,77.059998,77.059998,6800500
750,2018-12-26,300.000000,326.970001,294.089996,326.089996,326.089996,8163100,77.699997,82.800003,77.650002,82.800003,82.800003,10994700
751,2018-12-27,319.839996,322.170013,301.500000,316.130005,316.130005,8575100,81.330002,84.360001,80.239998,84.309998,84.309998,7849100
752,2018-12-28,323.100006,336.239990,318.410004,333.869995,333.869995,9939000,85.000000,85.230003,82.339996,83.260002,83.260002,6850400
